In [1]:
%load_ext autoreload 
%autoreload 2


In [2]:

#lb
import pandas as pd
edges = pd.concat([
    pd.read_parquet(f'/home/anhphantq/unidata_sub/edges_sub_frag_{i}_v1.parquet') for i in range (10) 
    ])
edges_test = pd.read_parquet('/home/anhphantq/unidata_sub/edges_test_sub_v1.parquet')

vertices = pd.read_parquet('/home/anhphantq/unidata_sub/train_vprops_v1.parquet')
vertices_test = pd.read_parquet('/home/anhphantq/unidata_sub/test_vprops_v1.parquet')

item_agg = pd.read_parquet('/home/anhphantq/unidata_sub/item_props_v1.parquet')
item_glob = pd.read_parquet('/home/anhphantq/unidata_sub/item_glob_v1.parquet')

In [2]:

#valid
import pandas as pd
edges = pd.concat([
    pd.read_parquet(f'/home/anhphantq/splitted_unidata_sub/edges_sub_frag_{i}_v1.parquet') for i in range (10) 
    ])
edges_test = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/edges_test_sub_v1.parquet')

vertices = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/train_vprops_v1.parquet')
vertices_test = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/test_vprops_v1.parquet')
item_agg = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/item_props_v1.parquet')
item_glob = pd.read_parquet('/home/anhphantq/splitted_unidata_sub/item_glob_v1.parquet')

In [3]:
MIN_TS = edges.iloc[:, 2].min()
MAX_TS = edges.iloc[:, 2].max()
NUM_TRAIN_EDGES = edges.shape[0]
MIN_TS, MAX_TS, NUM_TRAIN_EDGES

(0, 1814399, 94357806)

In [4]:
from xgb_utils import feature_id_map
import numpy as np 
# disk_vp = np.memmap('splitted_vertice_properties.array', dtype='float64', mode='r+').reshape(len(feature_id_map), -1)
disk_vp = np.fromfile('splitted_vertice_properties.array').reshape(len(feature_id_map), -1)
disk_vp.setflags(write = False)

In [4]:
from xgb_utils import feature_id_map
import numpy as np 
# disk_vp = np.memmap('vertice_properties.array', dtype='float64', mode='r+').reshape(len(feature_id_map), -1)
disk_vp = np.fromfile('vertice_properties.array').reshape(len(feature_id_map), -1)
disk_vp.setflags(write = False)

In [5]:
JOIN = True

In [5]:
JOIN = False

In [6]:
#Params
import numpy as np
TEST_CLICK_WEIGHT = 3
TEST_CART_WEIGHT = 6
TEST_ORDER_WEIGHT_L= 6
TEST_ORDER_WEIGHT_S = 2

MAX_NUM_CANDS = 100

MIN_TOPK_COUSER = 100
COFITNESS_THRES = 0


ALPHA = np.array([0.3, 0.5, 0.8]).reshape(-1,1)
BETA = 0.5
FILTER = False

#PARAMS

get_test_items_params = {
      'test_click_weight': TEST_CLICK_WEIGHT,
      'test_cart_weight': TEST_CART_WEIGHT,
      'test_order_weight_l': TEST_ORDER_WEIGHT_L,
      'test_order_weight_s': TEST_ORDER_WEIGHT_S,
      'day' : 3600 * 8,  
  }

get_items_params = {
      'click_weight': np.array([12,3,1]).reshape(-1,1),
      'cart_weight' : np.array([12, 6, 10]).reshape(-1,1), 
      'order_weight': np.array([6,1,1]).reshape(-1,1), 
      'day' : 3600 * 8,  
    }



# Create edges and vertices

In [7]:
first_test_session = edges_test.iloc[0,0]

In [8]:
all_edges = pd.concat((edges, edges_test))

In [9]:
del edges, edges_test

In [10]:
all_vertices = pd.concat((vertices, vertices_test))

In [11]:
del vertices, vertices_test

In [12]:
all_edges

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')","('subsession', 'encode')"
0,0,16246,62919,62919,1,0,0,2
1,0,30373,1798887,1798887,1,0,0,1099511627776
2,0,97836,1590509,1590509,1,0,0,137438953472
3,0,102416,1714839,1714839,1,0,0,549755813888
4,0,154930,1241409,1241409,1,0,0,134217728
...,...,...,...,...,...,...,...,...
5535985,12899774,33035,2419168,2419168,1,0,0,1
5535986,12899775,1743151,2419170,2419170,1,0,0,1
5535987,12899776,548599,2419172,2419172,1,0,0,1
5535988,12899777,384045,2419176,2419176,1,0,0,1


In [13]:
all_vertices

,"('session', '')","('ts', 'min')","('ts', 'max')","('ts', 'count')","('subsession', 'max')","('consistency', '')"
0,0,1659304800,1661103727,147,40,1798927
1,1,1659304800,1660857067,27,6,1552267
2,2,1659304800,1660577379,13,2,1272579
3,3,1659304800,1661109666,177,25,1804866
4,4,1659304800,1659304900,3,0,100
...,...,...,...,...,...,...
1801246,12899774,1661723968,1661723968,1,0,0
1801247,12899775,1661723970,1661723970,1,0,0
1801248,12899776,1661723972,1661723972,1,0,0
1801249,12899777,1661723976,1661723976,1,0,0


In [14]:
MAX_SESSION = all_vertices.iloc[:,0].max()  + 1
all_edges.iloc[:,1] = all_edges.iloc[:,1] + MAX_SESSION


In [15]:
all_vertices = all_vertices.values

In [16]:
import numpy as np
all_edges = all_edges.values

# Load to graph

In [17]:
import graph_tool

In [18]:
from graph_tool import Graph
ug = Graph(directed=False)

In [19]:
vmap = ug.add_edge_list(all_edges[:, : 2].astype(np.uint32))

In [20]:
ug.num_vertices(), ug.num_edges()

(14755382, 99893796)

In [21]:
#short type
all_edges[:, 4 : 7] = np.clip (all_edges[:, 4 : 7], 0,  32_767)

In [22]:
data_type = {
    'elts': 'unsigned int',
    'efts': 'unsigned int',
    'eclick': 'short',
    'ecart': 'short',
    'eorder': 'short',
    'esub': 'unsigned long'
}

In [23]:
for i, prop in enumerate(['elts', 'efts', 'eclick', 'ecart', 'eorder', 'esub']):
  setattr(ug.ep, prop, ug.new_ep(data_type[prop], vals = all_edges[:, i + 2]))  


In [24]:
del all_edges

In [25]:
elts = ug.ep['elts']
efts = ug.ep['efts']
eclick = ug.ep['eclick']
ecart = ug.ep['ecart']
eorder = ug.ep['eorder']
esub = ug.ep['esub']


In [26]:
vp = {}
vp['num_sub'] = ug.new_vp('double',  all_vertices[:, -2] ).get_array()
vp['consistency'] = ug.new_vp('double',  1 / (1 + np.log(all_vertices[:, -1] + 1)) ).get_array()
vp['num_actions'] = ug.new_vp('unsigned int', all_vertices[:, -3]).get_array()

In [27]:
del all_vertices

# Count actions

In [28]:
from graph_tool import incident_edges_op
vp['num_clicks'] = incident_edges_op(ug, "out", "sum", eclick).get_array()
vp['num_carts'] = incident_edges_op(ug, "out", "sum", ecart).get_array()
vp['num_orders'] = incident_edges_op(ug, "out", "sum", eorder).get_array()

# Get degree

In [29]:
vp['degree'] = ug.get_total_degrees(np.arange(ug.num_vertices()))

# Pagerank

In [30]:
%%time
import graph_tool.centrality as gt
vp['pr'] = gt.pagerank(ug).get_array()

CPU times: user 15min, sys: 524 ms, total: 15min 1s
Wall time: 1min


# Recent 7 day graph features

In [28]:
#remove item not interact in recent 7 days
efilter = (MAX_TS - elts.get_array() - 3600 * 24 * 7) < 0

In [29]:
from graph_tool import GraphView
fug = GraphView(ug, efilt = efilter)

In [33]:
vp['recent_num_clicks'] = incident_edges_op(fug, "out", "sum", eclick).get_array()
vp['recent_num_carts'] = incident_edges_op(fug, "out", "sum", ecart).get_array()
vp['recent_num_orders'] = incident_edges_op(fug, "out", "sum", eorder).get_array()
vp['recent_degree'] = fug.get_total_degrees(np.arange(fug.num_vertices()))
vp['recent_pr'] = gt.pagerank(fug).get_array()

# Item Features

In [34]:
item_agg

,aid,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,...,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2
0,0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,206.0,18.0,5.0,99.0,11.0,0.0,13.0,0.0,0.0,...,3.0,211.0,27.0,6.0,228.0,19.0,2.0,335.0,32.0,7.0
4,4,8.0,1.0,0.0,12.0,1.0,0.0,7.0,0.0,0.0,...,0.0,8.0,1.0,0.0,10.0,0.0,0.0,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839553,1855598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1839554,1855599,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1839555,1855600,2.0,0.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,...,0.0,4.0,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0
1839556,1855601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
item_index = item_agg.aid.values + MAX_SESSION

In [36]:
for i, col in enumerate(item_agg.columns[1:]):
    val = np.zeros(ug.num_vertices(), dtype= int)
    val[item_index] = item_agg.values[:, 1 + i]
    vp[col] = val 

In [37]:
del item_agg, item_index

# Item Glob Features

In [38]:
item_glob

,"('aid', '')","('scaled_ts', 'max')","('scaled_ts', 'min')","('time_decay', 'sum')","('click', 'sum')","('cart', 'sum')","('order', 'sum')"
0,0,20.863345,1.452789,13.572334,13.572334,0.000000,0.000000
1,1,17.508866,2.738449,7.465299,7.319284,0.146015,0.000000
2,2,19.611991,4.524248,6.230244,6.230244,0.000000,0.000000
3,3,20.984317,0.434826,1461.256950,1322.898875,115.314236,23.043839
4,4,20.898519,0.388796,79.325614,75.313527,4.012088,0.000000
...,...,...,...,...,...,...,...
1839553,1855598,17.944850,0.741273,2.034551,2.034551,0.000000,0.000000
1839554,1855599,19.679792,3.639051,4.168844,4.168844,0.000000,0.000000
1839555,1855600,20.814815,1.492014,27.600753,25.150745,2.321962,0.128045
1839556,1855601,12.285602,0.274120,7.111232,6.075555,1.035678,0.000000


In [39]:
item_index = item_glob.iloc[:,0].values + MAX_SESSION

In [40]:
from xgb_utils import glob_features

In [41]:
for i, col in enumerate(glob_features[:-1]):
    val = np.zeros(ug.num_vertices(), dtype= float)
    val[item_index] = item_glob.values[:, 1 + i]
    vp[col] = val 


In [42]:
vp['item_glob_durability']= np.zeros(ug.num_vertices(), dtype= float)
vp['item_glob_durability'][item_index] = item_glob.iloc[:,1].values - item_glob.iloc[:,2].values


In [43]:
del item_glob, item_index


# Create Disk VP

In [44]:
vp.keys()

dict_keys(['num_sub', 'consistency', 'num_actions', 'num_clicks', 'num_carts', 'num_orders', 'degree', 'pr', 'recent_num_clicks', 'recent_num_carts', 'recent_num_orders', 'recent_degree', 'recent_pr', 'recent_day7_type0', 'recent_day7_type1', 'recent_day7_type2', 'recent_day6_type0', 'recent_day6_type1', 'recent_day6_type2', 'recent_day5_type0', 'recent_day5_type1', 'recent_day5_type2', 'recent_day4_type0', 'recent_day4_type1', 'recent_day4_type2', 'recent_day3_type0', 'recent_day3_type1', 'recent_day3_type2', 'recent_day2_type0', 'recent_day2_type1', 'recent_day2_type2', 'recent_day1_type0', 'recent_day1_type1', 'recent_day1_type2', 'item_glob_last_action', 'item_glob_first_action', 'item_glob_time_decay_sum', 'item_glob_time_decay_sum_click', 'item_glob_time_decay_sum_cart', 'item_glob_time_decay_sum_order', 'item_glob_durability'])

In [45]:
shape = None
for k in vp.keys():
  vp[k] = np.array(vp[k])
  if shape:
    assert vp[k].shape == shape, (k, vp[k].shape, shape)
  else:
    shape = vp[k].shape

In [46]:
import multiprocessing
mem_size = (len(vp.keys()), vp[k].shape[0])
mem_size

(41, 16427185)

In [49]:
disk_vp = np.memmap('splitted_vertice_properties.array', dtype='float64', mode='w+', shape=(mem_size[0],mem_size[1]))


In [47]:
disk_vp = np.memmap('vertice_properties.array', dtype='float64', mode='w+', shape=(mem_size[0],mem_size[1]))


In [48]:
for i, k in enumerate(vp.keys()):
  disk_vp[i] = vp[k] 


In [49]:
del vp

# Load VP

In [28]:
assert disk_vp.shape[1] == ug.num_vertices(), (disk_vp.shape[1], ug.num_vertices())

# Load test

In [28]:
ROOT = '/home/anhphantq/otto'
DATA_FOLDER = ROOT + '/data'



In [29]:
ROOT = '/home/anhphantq/otto'
DATA_FOLDER = ROOT + '/splitted_data'



In [30]:
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')

In [31]:
import numpy as np
session_list = np.sort(test_df['session'].unique())


In [32]:
session_list = session_list[:len(session_list) // 20]

In [31]:
session_list = session_list[len(session_list) // 20 : len(session_list) // 10]


In [32]:
# session_list = session_list[len(session_list) // 20 : ]

In [33]:
len(session_list)

90062

In [34]:
del test_df

# Find neighbour strategy

In [35]:
from xgb_utils import USER_FEATURES, ITEM_FEATURES, \
feature_id_map, recent_features_id_map, recent_features, \
glob_features_id_map, glob_features

user_feature_ids = [feature_id_map[f] for f in USER_FEATURES]
all_item_feature_ids = [feature_id_map[f] for f in ITEM_FEATURES]

recent_feature_ids = [recent_features_id_map[f] for f in recent_features]
glob_feature_ids = [glob_features_id_map[f] for f in glob_features]
len(user_feature_ids), len(all_item_feature_ids)

(7, 38)

In [36]:

def get_user_features(user):
  return [disk_vp[feature_id_map[f]][user] for f in USER_FEATURES]

def get_item_features(items, id_list):

  return [
      np.array([
          disk_vp[i][item] for item in items 
      ]) for i in id_list
  ]


def get_test_items(user, beta):
  global ug, fug, elts, eclick, ecart, eorder, get_test_items_params, FILTER
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder 6 subsession, 7 ftss
  edges = ug.get_all_edges(user, eprops = [elts, eclick, ecart, eorder, esub, efts])
        
  return _get_test_items(edges, beta, **get_test_items_params)


def get_user_neighbours(item):
  global ug, fug, elts, efts, eclick, ecart, eorder, esub, FILTER
  #1 target 2 ltss 3 ftss 4 click 5 cart 6 order 7 subsession
  edges = ug.get_all_edges(item, eprops = [elts, efts, eclick, ecart, eorder, esub])
  return edges[:,1], edges[:,2], edges[:,3], edges[:, 4], edges[:, 5], edges[:, 6], edges[:, 7]
  

def get_item_neighbours(user, lts, fts, prevsub, prev_click, prev_cart, prev_order, alpha):
  global ug, fug, elts, efts, eclick, ecart, eorder, esub, get_items_params, FILTER
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order, 7 subsession

  edges = ug.get_all_edges(user, eprops = [elts, efts, eclick, ecart, eorder, esub])

  return _get_item_neighbours(edges, lts, fts, prev_click = prev_click, prev_cart = prev_cart, prev_order = prev_order
                              , alpha = alpha, prevsub = prevsub, **get_items_params)

def get_popular_in_neighbourhood(neighbourhood, exclude):
    
    edges = np.concatenate([ug.get_all_edges(user, eprops = [eclick, ecart, eorder])[:, 1:]
                           for user in neighbourhood])
    return sum_by_group_exclude_multiple_values_return_num_appearance(edges[:,0], np.transpose(edges[:, 1 :]), exclude)


# Numba funcs

In [37]:
import numba as nb
import numpy as np
from numba import prange
    
@nb.njit(nb.float64[:](nb.int64[:]), cache = True, nogil = True)
def reverse_normalize(X):
  m = np.max(X)
  return (m - X) / (m - np.min(X) + 1e-12) 

@nb.njit(nb.float64[:](nb.int64[:]), cache = True, nogil = True)
def normalize(X):
  m = np.max(X)
  return (X - np.min(X)) / (m - np.min(X) + 1e-12) 

@nb.njit([nb.int64[:](nb.float64[:]),
          nb.int64[:](nb.int64[:]),
          ], cache = True, nogil = True)
def get_rank(x):
  return np.argsort(np.argsort(x))

@nb.njit(nb.int64(nb.int64), cache = True, nogil = True)
def decode_1d(x):
  rs = 0
  while x > 0:
    if x % 2 == 1:
      rs += 1
    x = x // 2
  return rs

@nb.njit(nb.int64[:](nb.int64[:]), cache = True, nogil = True)
def decode_2d(x):
  return np.array([decode_1d(i) for i in x])



@nb.njit([
  nb.float64[:, :](nb.float64[:,:]),
  nb.int64[:, :](nb.int64[:,:]),
          ])
def cumsum_2d(x):
  result = np.empty_like(x)
  result[:, 0] = x[:, 0]
  for i in prange(1, x.shape[1]):
    result[:, i] = x[:, i] +  result[:, i - 1]
  return result


@nb.njit([nb.float64[:](nb.float64[:], nb.float64, nb.float64),
          nb.float64[:](nb.float64[:], nb.int64, nb.float64),
          nb.float64[:](nb.float64[:], nb.float64, nb.int64),
         nb.float64[:](nb.float64[:], nb.int64, nb.int64)], cache = True, nogil = True)
def normalize_to_range(X, l, r):
  m = np.min(X)
  return (X - m) / (np.max(X) - m + 1e-12) * (r -l) + l


@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:], nb.int64[:])) (nb.int64[:], nb.float64[:,:]),
          nb.types.Tuple((nb.int64[:], nb.int64[:,:], nb.int64[:])) (nb.int64[:], nb.int64[:,:]),
          ], cache = True, nogil = True)
def sum_by_group_multiple_values_return_num_appearance(groups, values):
    #values shape n_values * n_items
    order = np.argsort(groups)

    groups = groups[order]
    values = values[:, order]
    values = cumsum_2d(values)

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))

    values = values[:, index]

    groups = groups[index]
    values[:, 1:] = values[:, 1:] - values[:, :-1]
    return groups, values, nodes[1:] - nodes[:-1]



@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:, :], nb.int64[:])) (nb.int64[:], nb.float64[:,:], nb.int64[:]),
          nb.types.Tuple((nb.int64[:], nb.int64[:, :], nb.int64[:])) (nb.int64[:], nb.int64[:,:], nb.int64[:]),
          ], cache = True, nogil = True)
def sum_by_group_exclude_multiple_values_return_num_appearance(groups, values, exclude):
    #remove existed items
    inclusive = np.ones(len(groups), 'bool')
    for e in exclude:
      inclusive = inclusive & (groups != e)
    #values shape n_values * n_items
    return  sum_by_group_multiple_values_return_num_appearance(groups[inclusive], values[:, inclusive])

    
@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:,:])) (nb.int64[:], nb.float64[:,:,:]),
          ], cache = True, nogil = True)
def sum_by_group_multiple_values_2d(groups, values):
    order = np.argsort(groups)

    groups = groups[order]
    #first axis of values  is 3 types of actions , second axisis heterogenous, third axis is different items
    values = values[:, :, order]
    for i in prange(values.shape[0]):
      values[i] = cumsum_2d(values[i])

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]

    values = values[:, :, index]

    groups = groups[index]
    values[:, :, 1:] = values[:, :,1:] - values[:, :,:-1]

    return groups, values

@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:,:])) (nb.int64[:], nb.float64[:,:,:]),
          ], cache = True, nogil = True)
def max_by_group_multiple_values_2d(groups, values):
    order = np.argsort(groups)

    groups = groups[order]
    #first axis of values  is 3 types of actions , second axisis heterogenous, third axis is different items
    values = values[:, :, order]
    

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    groups = groups[index]
    ends = np.where(index)[0]

    k = 0
    for i in prange(1, values.shape[2]):
        if i != ends[k] + 1:
            values[:, :, i] = np.maximum(values[:, :, i], values[:, :, i - 1])
        else:
            k+=1
    
    values = values[:, :, index]

    return groups, values

@nb.njit(nb.types.Tuple((nb.int64[:], nb.float64[:],
                         nb.int64[:], nb.int64[:],
                         nb.int64[:], nb.int64[:],
                         nb.float64[:], nb.float64[:],
                         nb.float64[:], nb.int64[:],
                         nb.int64[:]))(nb.int64[:, :], nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.int64) , cache = True, nogil = True)
def _get_test_items(edges, beta, test_click_weight, test_cart_weight, test_order_weight_l, test_order_weight_s, day):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder 6:susbsession, 7 fts
  
  time_decay = beta ** reverse_normalize(edges[:, 2])

  order_weight = np.where(edges[:, 5] > 1, test_order_weight_l, test_order_weight_s)

  weight = edges[:, 3] * test_click_weight + edges[:, 4] * test_cart_weight + order_weight 

  fitness =  time_decay * weight 

  order = np.argsort(-fitness)

  num_subsession = decode_2d(edges[ :, 6])
  return edges[order, 1], fitness[order], edges[order, 3] , edges[order, 4], edges[order, 5], \
          num_subsession[order], time_decay[order], normalize(edges[order,2]), normalize(edges[order, 7]), edges[order, 2] - edges[order, 7], edges[order, 3] + edges[order, 4] + edges[order, 5]
          

@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.float64[:], nb.int64, nb.float64),
          nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.int64[:], nb.int64, nb.float64),
          # nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.float64[:], nb.float64[:], nb.int64, nb.float64),
          ], cache = True, nogil = True)
def find_topk_user(cousers, cofitness, min_topk_user, cofitness_thres):
  #sort cousers ascending by id
  order = np.argsort(cousers)
  cousers = cousers[order]
  cofitness = cofitness[order]

  #calculate groupby sum of cousers
  cumcofitness = np.cumsum(cofitness)

  #find unique
  index = np.ones(len(cousers), 'bool')

  #this is the last index of each group
  index[:-1] = cousers[1:] != cousers[:-1]
  cumcofitness = cumcofitness[index]
  

  #find start and end of groups
  nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))

  starts = nodes[:-1]
  ends = nodes[1:]
  lens = ends - starts
  rank_nums = get_rank(lens)
    
  #caluculate groupby sum of cousers
  cumcofitness[1:] = cumcofitness[1:] - cumcofitness[:-1]

  rank_fit = get_rank(cumcofitness)
  
  order_fit = np.argsort(-(rank_nums + rank_fit))

  k = min_topk_user
  if cofitness_thres > 0:
    #get topk user
    #dynamic top user
    total_fitness = np.sum(cumcofitness)
    
    k = 0
    remain = 0
    thresh_hold = cofitness_thres * total_fitness
    while remain < thresh_hold:
      remain += cumcofitness[order_fit][k]
      k+=1 
    
    if k < min_topk_user:
      k = min_topk_user

    if k > 500:
      k = 500

  
  order_fit = order_fit[:k]

  starts = nodes[order_fit]
  ends = nodes[order_fit + 1]
  
  filter = np.zeros(len(cousers), 'bool')
  for i in prange(len(starts)):
    filter[starts[i] : ends[i]] = 1
    cofitness[starts[i] : ends[i]] = cofitness[starts[i] : ends[i]] * (ends[i] - starts[i])
  return cousers[filter], cofitness[filter], order, filter

In [38]:
@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:,:, :]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.float64, nb.int64, nb.float64[:, :], nb.float64[:,:], nb.float64[:,:] , nb.float64[:,:], nb.int64), 
          nb.types.Tuple((nb.int64[:], nb.float64[:,:, :]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.int64,   nb.int64, nb.float64[:, :], nb.int64[:,:], nb.int64[:,:] , nb.int64[:,:], nb.int64), 
          ], cache = True, nogil = True)
def _get_item_neighbours(edges, lts, fts, prevsub, prev_click, prev_cart, prev_order, alpha, click_weight, cart_weight, order_weight, day):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order, 7 -> subsession

  cosubsession = prevsub & edges[:, 7]
  cosubsession = decode_2d(cosubsession)

  reverse_user_variety = 1  / (1 + np.log(np.sum(decode_2d(edges[:, 7])) + 1))
  reverse_user_consistency = 1 / (1 + np.log((np.amax(edges[:,2]) - np.amin(edges[:, 3])) + 1))

  coclick = np.minimum(edges[:, 4], prev_click)
  cocart = np.minimum(edges[:, 5], prev_cart)
  coorder = np.minimum(edges[:, 6], prev_order)

  #weight 3 x n, line 0 find click, 1 find cart, 2 find order
  weight = coclick * click_weight + cocart * cart_weight  + coorder * order_weight 

  #check if union, if yes time diff = 0, else = min(fts1 - lts2, fts2 - lts1)
  is_end_later = edges[:, 2] > lts

  is_union = np.empty(edges.shape[0], dtype = np.bool_)
  is_union[is_end_later] =  edges[is_end_later, 3] < lts
  is_end_sooner = ~is_end_later
  is_union[is_end_sooner] = edges[is_end_sooner, 2] > fts

  
  subsession_coef = np.clip(1.8 ** cosubsession, 0, 1)

  time_diff = np.empty(edges.shape[0],  dtype = np.float32) 
  
  time_diff[is_union] = 0

  not_union_and_later = ~is_union & is_end_later
  not_union_and_sooner = ~is_union & is_end_sooner


  time_diff[not_union_and_later] = np.abs(lts - edges[not_union_and_later, 3]) / day
  time_diff[not_union_and_sooner] = np.abs(fts - edges[not_union_and_sooner, 2]) / day
  time_decay = alpha ** time_diff
  
  

  fitness_subco = weight  * subsession_coef * reverse_user_variety * reverse_user_consistency 

  fitness_time_decay = weight * time_decay  * reverse_user_variety * reverse_user_consistency 

  #shape 3 x num_values x num_items
  values_to_return = np.stack((
      fitness_subco, fitness_time_decay, np.stack((cosubsession,cosubsession,cosubsession)),
      np.stack((coclick,coclick,coclick)), np.stack((cocart,cocart,cocart)), np.stack((coorder,coorder,coorder))
  ), axis = 1)
  
  return sum_by_group_multiple_values_2d(edges[:, 1], values_to_return)

# Step

In [109]:
def get_type_items(user, clicks, carts, orders, lens, rerank_tracking, train_data_clicks= None, train_data_carts= None, train_data_orders= None, num_cands = None,
                   test_train_data = None, test_num_cands = None, get_train = False):

  global ALPHA, BETA, MIN_TOPK_COUSER, COFITNESS_THRES, XGBOOST, GET_TEST_TRAIN, XGBOOST_TEST
  #get user features
  if XGBOOST or get_train or GET_TEST_TRAIN: 
    ori_user_features = get_user_features(user)
  
  #fitness sorted descending
  items, fitness, num_clicks, num_carts, num_orders, \
   num_subsession, time_decay, lts, fts, consistency, num_interacts = get_test_items(user, BETA)

  if GET_TEST_TRAIN:
    n = n = min(MAX_NUM_CANDS, len(items))
    if n > 20 or (not XGBOOST):
      
      user_features = []
      
      for f in ori_user_features:
        user_features.append(np.full(n, f))

      item_features = get_item_features(items[:n], all_item_feature_ids)

      test_train_data.append(np.transpose(np.stack((np.full(n, user),
                                          items[:n],
                                          fitness[:n],
                                          num_clicks[:n],
                                          num_carts[:n],
                                          num_orders[:n],
                                          num_subsession[:n],
                                          time_decay[:n],
                                          lts[:n],
                                          fts[:n],
                                          consistency[:n],
                                          num_interacts[:n],
                                          *user_features, 
                                          *item_features, 
                                          ))))
      test_num_cands.append(np.full(n, n))
  

  lens.append(len(items))
  #level 2 rerank
  need_rerank = False

  if len(items)  > 19:
    clicks.append(items[:20].tolist())
    carts.append(items[:20].tolist())
    orders.append(items[:20].tolist())

  else:
    num_more = 20 - len(items)
    
    
    
    #get couser
    coalls = [get_user_neighbours(items[i]) for i in range(len(items))]    
    cofitness = np.concatenate([np.full_like(all[0], fitness[i]) for i, all in enumerate(coalls)] )

    #find topk cousers
    cousers, cofitness, order, filter  = find_topk_user(np.concatenate([all[0] for all in coalls]), cofitness, MIN_TOPK_COUSER, COFITNESS_THRES)
    coltss = np.concatenate([all[1] for all in coalls])[order][filter]
    coftss = np.concatenate([all[2] for all in coalls])[order][filter]
    coclick = np.concatenate([all[3] for all in coalls])[order][filter]
    cocart = np.concatenate([all[4] for all in coalls])[order][filter]
    coorder = np.concatenate([all[5] for all in coalls])[order][filter]
    cosubsession = np.concatenate([all[6] for all in coalls])[order][filter]

    

    #get coitems
    coalls = [get_item_neighbours(cousers[i], coltss[i], coftss[i], cosubsession[i], coclick[i], cocart[i], coorder[i], ALPHA) for i in range(len(cousers))]
    coitems = np.concatenate([all[0] for all in coalls])
    
    #get popular items in user neighbourhood
    neighbourhood = np.unique(cousers)    
    popular_items, popular_attrs, popular_num_appearance = get_popular_in_neighbourhood(neighbourhood, exclude =  items)  
    popular_attrs = [pa for pa in popular_attrs]
    popular_attrs.append(popular_num_appearance)
    

    click_rank_objs = list(zip(popular_attrs, [1,6,3,2]))
    cart_rank_objs = list(zip(popular_attrs, [1,6,3,2]))
    order_rank_objs = list(zip(popular_attrs, [1,6,3,2]))
    rank_objs = [click_rank_objs, cart_rank_objs, order_rank_objs]
    
    #get each action type prediction
    for j, (a, train_data_a) in enumerate(zip([clicks, carts, orders], [train_data_clicks, train_data_carts, train_data_orders])):

      #multiply item by fitness of user which interacted with it
      
      this_action_cofitness = np.concatenate([all[1][j]* cofitness[i] for i, all in enumerate(coalls)], axis = -1) 
      
      
      #count covisit
      this_action_coitems, this_action_cofitness, num_appearance= sum_by_group_exclude_multiple_values_return_num_appearance(coitems, this_action_cofitness, exclude =  items)
      
      cofitness_cosub, cofitness_time_decay, \
      num_cosub,\
      coclick, cocart, coorder = (this_action_cofitness[i] for i in range(this_action_cofitness.shape[0]))
      
      penalty = 1
      
      # knowledge from xgboost 
      if j in [1, 2]:
        degrees, recent_day1_type1 = get_item_features(this_action_coitems, [feature_id_map['degree'], feature_id_map['recent_day1_type1']])
        
        penalty = penalty * np.where(degrees > 300, 0.5, 1)
        penalty = penalty * np.where(recent_day1_type1 > 1, 2, 1)
      
      rank_cosub = get_rank(cofitness_cosub * penalty) * 10
      rank_time_decay = get_rank(cofitness_time_decay * penalty) * 10
      rank_num_appearance = get_rank(num_appearance) * 2

      
      rank_sum = rank_cosub  + rank_time_decay + rank_num_appearance
      
      order = np.argsort(-rank_sum)
            
      
      assert popular_items.shape[0] == this_action_coitems.shape[0]
            
      if get_train:
        
        if len(this_action_coitems):
          if j == 0:
            n = min(MAX_NUM_CANDS, len(this_action_coitems))
            num_cands.append(np.full(n, n))
            need_rerank = True


          aggs = ['mean', 'var']
          features = [cofitness_cosub[order[:n]],
                      cofitness_time_decay[order[:n]],
                      num_appearance[order[:n]],
                      num_cosub[order[:n]],
                      coclick[order[:n]],
                      cocart[order[:n]],
                      coorder[order[:n]]]
          
          
          #quotient features
          quo_features = []
          agg_features = []
          for f in features:
            for agg in aggs: 
              agg_val = getattr(f,agg)()
              agg_features.append(np.full(n, agg_val))
              quo_features.append(f / (agg_val + 1))
              
          
          
          user_features = []
          
          for f in ori_user_features:
            user_features.append(np.full(n, f))

          item_features = get_item_features(this_action_coitems[order[:n]], all_item_feature_ids)
                
          train_data_a.append(np.transpose(np.stack((np.full(n, user),
                                                  this_action_coitems[order[:n]],
                                                  np.full(n, j), #action
                                                  *features,
                                                  *item_features, 
                                                  *[pa[order[:n]] for pa in popular_attrs],
                                                  *quo_features,
                                                  *agg_features,
                                                  *user_features,
                                                  ))))
          

      if XGBOOST: 
        # news = items.tolist() +  coitems[order[:num_more]].tolist() 
        # a.append(news)
        a.append(items.tolist())
      else:
        news = items.tolist() +  this_action_coitems[order[:num_more]].tolist() 
        a.append(news)
  rerank_tracking.append(need_rerank)
    
    

    


# Batchproc and XGBoost postprocess


In [77]:
import multiprocessing
import xgboost as xgb

def batch_proc(*args):
  global GET_TRAIN, XGBOOST, GET_RESULT
  batch, batch_name = args[0]
  #create a random 0 to prevent numpy cast clicks into 2d array instead of 1d array of list 
  clicks = [0]
  carts = [0]
  orders = [0]
  lens = []
  num_cands = []
  rerank_tracking = []
  train_data_clicks = []
  train_data_carts = []
  train_data_orders = []
  test_num_cands = []
  test_train_data = []
  for user in batch:
    get_type_items(user, clicks, carts, orders, lens, rerank_tracking, train_data_clicks, train_data_carts, train_data_orders, num_cands, test_train_data, test_num_cands, GET_TRAIN)

  if XGBOOST:
    clicks = np.array(clicks, dtype='object')
    carts = np.array(carts, dtype='object')
    orders = np.array(orders, dtype='object')
  
  #remove the initial blank list
  clicks = clicks [1:]
  carts = carts[1:]
  orders = orders[1:]

  result = []
  if GET_RESULT:
    result.append( [[clicks, carts, orders, lens], CACHE + f'/result_batch_{batch_name}'])
    
  if GET_TRAIN:
    train_data = []
    for data in [train_data_clicks, train_data_carts, train_data_orders]:
      train_data.extend(data)
      
    result.append([ [ train_data, num_cands], CACHE + f'/train_data_{batch_name}'])
  
  if GET_TEST_TRAIN:
    result.append( [ [test_train_data, test_num_cands], CACHE + f'/test_train_data_{batch_name}'])
  
  if XGBOOST:
    result.append(np.array(rerank_tracking))
  return result, batch
    

In [41]:
import traceback
from xgb_utils import xgboost_infer, xgboost_test_infer

def postprocess(rs, batch, test = False):
  if XGBOOST: 
    result_file = rs[0][1]
    clicks, carts, orders, lens = rs[0][0]
    try:
      infer_data, num_cands = rs[1][0]
      rerank_tracking = rs[3]
      assert len(infer_data) / 3 == rerank_tracking.sum(), (len(infer_data), len(rerank_tracking))
      num_cands = np.concatenate(num_cands)
      num_features = infer_data[0].shape[-1]

      infer_data = np.concatenate(infer_data).reshape(3, -1, num_features)
      infer_data = [infer_data[i] for i in range(3)]
    except:
      pass
      print(traceback.format_exc())

    over_lens = np.where(np.array(lens) > 20)[0]
    infer_test_data, test_num_cands = rs[2][0]
    assert len(infer_test_data) == len(over_lens), (len(infer_test_data), len(over_lens))
    num_features = infer_test_data[0].shape[-1]
    infer_test_data = np.concatenate(infer_test_data).reshape(-1, num_features)
    #rerank test items
    
    if len(over_lens): 
      user = batch [over_lens] 
      p = np.argsort(user)
      suggest = [clicks[over_lens], carts[over_lens], orders[over_lens]]

      for i, models in enumerate([test_click_models, test_cart_models, test_order_models]):
        rerank, u = xgboost_test_infer(infer_test_data, p, models)

        assert np.array_equal(u, user), (u[:10], user[:10])
        for j in range(len(suggest[i])):
          suggest[i][j] = rerank[j][:20]

      clicks[over_lens] = suggest[0]
      carts[over_lens] = suggest[1]
      orders[over_lens] = suggest[2]
      

    #rerank level 2 items
    # if False:
    if len(rerank_tracking):
      assert len(clicks.shape) == 1
      assert len(rerank_tracking.shape) == 1, len(rerank_tracking.shape)

      user = batch[rerank_tracking]
      p = np.argsort(user)
      
      suggest = [clicks[rerank_tracking], carts[rerank_tracking], orders[rerank_tracking]]

      for i, (data, models) in enumerate(zip(infer_data, [click_models, cart_models, order_models])):
        rerank, u = xgboost_infer(data, p, models)

        assert np.array_equal(u, user), (u[:10], user[:10])
        for j in range(len(suggest[i])):
          suggest[i][j] = (suggest[i][j] +  rerank[j])[:20]

      rerank_tracking = np.where(rerank_tracking)[0]
      clicks[rerank_tracking] = suggest[0]
      carts[rerank_tracking] = suggest[1]
      orders[rerank_tracking] = suggest[2]

    if not test:
      with open(result_file, 'wb') as f: 
        pickle.dump([clicks.tolist(), carts.tolist(), orders.tolist(), lens], f)

  if WRITE_FILE and (not test):
    
    for data, fn in rs: 
        with open(fn, 'wb') as f: 
          pickle.dump(data, f)

# Infer Params

In [42]:
import multiprocessing
CACHE = '/home/anhphantq/vdb/cache_valid'

MODEL_PATH = '/home/anhphantq/xgboost'
MODEL_VERSION = '12'
TEST_MODEL_PATH = '/home/anhphantq/xgboost_test'
TEST_MODEL_VERSION = '8'

#TO use xgboost enable get_train, get_result, get_test_train and disable writefile
GET_TRAIN = False
GET_TEST_TRAIN = False
GET_RESULT = True
WRITE_FILE = True
XGBOOST = False



NUM_CPU = multiprocessing.cpu_count()
BATCH_SIZE = 1400
# BATCH_SIZE = 4500

In [43]:
!rm -rf {CACHE}
!mkdir {CACHE}

# Create data batches

In [44]:
from utils import shuffle, unshuffle

In [45]:
import _pickle as pickle
perm = shuffle(session_list)
with open(CACHE + '/perm', 'wb') as f:
  pickle.dump(perm, f)

In [46]:
import _pickle as pickle

#divide into batches

num_batches = len(session_list) // BATCH_SIZE

start = np.arange(num_batches) * BATCH_SIZE
end = start + BATCH_SIZE
end[-1] = len(session_list)
batches = [session_list[start[i] : end[i]] for i in range(len(start))]
batch_names = [i for i in range(len(batches))]

with open(CACHE + '/metadata', 'wb') as f:
  pickle.dump({
      'num_frags': len(batches), 
      'max_session': MAX_SESSION,
  }, f)

# Load XGB models

In [46]:
import xgboost as xgb
click_models = []
for fold in range(3):
  click_model = xgb.Booster()
  click_model.load_model(MODEL_PATH + '/clicks_only_' + MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  click_models.append(click_model)

cart_models = []
for fold in range(3):
  cart_model = xgb.Booster()
  cart_model.load_model(MODEL_PATH + '/carts_only_'+ MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  cart_models.append(cart_model)

order_models = []
for fold in range(3):
  order_model = xgb.Booster()
  order_model.load_model(MODEL_PATH + '/orders_only_'+ MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  order_models.append(order_model)

In [47]:
import xgboost as xgb
test_click_models = []
for fold in range(3):
  test_click_model = xgb.Booster()
  test_click_model.load_model(TEST_MODEL_PATH + '/clicks_only_' + TEST_MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  test_click_models.append(test_click_model)

test_cart_models = []
for fold in range(3):
  test_cart_model = xgb.Booster()
  test_cart_model.load_model(TEST_MODEL_PATH + '/carts_only_'+ TEST_MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  test_cart_models.append(test_cart_model)

test_order_models = []
for fold in range(3):
  test_order_model = xgb.Booster()
  test_order_model.load_model(TEST_MODEL_PATH + '/orders_only_'+ TEST_MODEL_VERSION + f'/XGB_fold{fold}.xgb')
  test_order_models.append(test_order_model)

# Test pipeline

In [114]:
from tqdm import tqdm
clicks = []
carts = []
orders = []
lens = []
train_data =[]
num_cands = []
# for user in tqdm(np.random.choice(session_list, size = 1000)):
rs, batch = batch_proc((batches[0][:100], [0]))
postprocess(rs, batch, test = True) 


# Execute Multiprocessing

In [110]:
%%time
from tqdm import tqdm
import _pickle as pickle

pool = multiprocessing.Pool(NUM_CPU)

for rs, batch in tqdm(pool.imap_unordered(batch_proc, tuple(zip(batches, batch_names)), chunksize = 1), total = num_batches):
  postprocess(rs, batch)



100%|██████████| 64/64 [00:50<00:00,  1.27it/s]

CPU times: user 1.93 s, sys: 9.26 s, total: 11.2 s
Wall time: 59.3 s


In [111]:
pool.close()
pool.join()


# Load result from pickle

In [112]:
CACHE = '/home/anhphantq/vdb/cache_valid'


In [113]:
from utils import unshuffle

In [114]:
import _pickle as pickle
with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = metadata['num_frags']
MAX_SESSION = metadata['max_session']

In [115]:
import multiprocessing
from tqdm import tqdm

clicks = []
carts = []
orders = []
lens = []
for i in tqdm(range(num_frags)):
    with open(CACHE + f'/result_batch_{i}', 'rb') as f:
      result = pickle.load(f)
    
    clicks.extend(result[0])
    carts.extend(result[1])
    orders.extend(result[2])
    lens.extend(result[3])
    


100%|██████████| 64/64 [00:00<00:00, 79.38it/s] 


In [116]:
del result

In [117]:
import _pickle as pickle
with open(CACHE + '/perm', 'rb') as f:
  perm = pickle.load(f)
# unshuffle([clicks,carts,orders,lens, session_list] + [train_data] if GET_TRAIN else [], perm)
unshuffle([clicks,carts,orders,lens, session_list], perm)
# unshuffle([clicks,carts,orders], perm)

# Create submission

In [118]:
import pandas as pd
from utils import create_submission
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

(816, 9252, 270186)

In [119]:
final = create_submission(clicks, carts, orders, session_list, MAX_SESSION, JOIN)

In [120]:
final

,session_type,labels
0,11172317_clicks,"[1280103, 802650, 479009, 58186, 1153899, 9840..."
1,11138029_clicks,"[689567, 316498, 1740927, 1083413, 496180, 119..."
2,11171259_clicks,"[28120, 102466, 51244, 1542473, 468342, 87613,..."
3,11171967_clicks,"[496180, 861401, 688179, 440836, 1755739, 1162..."
4,11157433_clicks,"[590833, 1533875, 190430, 28092, 694140, 10804..."
...,...,...
90057,11104763_orders,"[1007548, 1420240, 208189, 507852, 407533, 157..."
90058,11119471_orders,"[1490374, 486254, 1587604, 1272191, 297789, 16..."
90059,11182922_orders,"[586206, 202774, 883073, 199409, 271388, 22641..."
90060,11119194_orders,"[30107, 311422, 1589974, 1586160, 166037, 1651..."


# Eval valid

In [121]:
from utils import get_score

In [122]:
get_score(final, session_list, JOIN, n = 20)

Score : 0.5459015086718482


type
carts     0.389250
clicks    0.507891
orders    0.630562
dtype: float64

# Save sumission

In [ ]:
final.to_csv('submission.csv', index = False)

#Submit


In [ ]:
!kaggle competitions submit -c otto-recommender-system -f submission.csv -m "hehe"

100%|████████████████████████████████████████| 782M/782M [00:37<00:00, 21.9MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System